## 1 Bias and variance of ridge regression

## 2 Denoising of a CT image

## 3 Automatic feature selection for regression

### 3.1 Implement Orthogonal Matching Pursuit

### 3.2 Classification with sparse LDA

### 3.3 One-against-the-rest classication